In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import numpy as np
import pandas as pd
import nltk
import regex as re
from tqdm import tqdm
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer,lancaster
from sklearn.naive_bayes import ComplementNB , MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
df = pd.read_csv(r'data.csv')

In [3]:
df.head()

,source,review_description,rating,review_date
0,Google Play,Meh. Not the greatest experience on a Chromebo...,2,2023-07-08 14:18:24
1,Google Play,Pretty good for a first launch!! Its easy to u...,3,2023-07-19 20:52:48
2,Google Play,"For a brand new app, it's very well optimized....",3,2023-07-06 23:03:11
3,Google Play,"Great app with a lot of potential! However, th...",3,2023-07-10 00:53:25
4,Google Play,"The app is good, but it needs a lot of functio...",3,2023-07-06 16:57:43


In [4]:
df.shape

(32910, 4)

In [5]:
df.isnull().sum()

source                0
review_description    0
rating                0
review_date           0
dtype: int64

In [6]:
df.duplicated().sum()

1

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
stp=stopwords.words('english')
lm=WordNetLemmatizer()
stp.remove('not')

In [9]:
df.head()

,source,review_description,rating,review_date
0,Google Play,Meh. Not the greatest experience on a Chromebo...,2,2023-07-08 14:18:24
1,Google Play,Pretty good for a first launch!! Its easy to u...,3,2023-07-19 20:52:48
2,Google Play,"For a brand new app, it's very well optimized....",3,2023-07-06 23:03:11
3,Google Play,"Great app with a lot of potential! However, th...",3,2023-07-10 00:53:25
4,Google Play,"The app is good, but it needs a lot of functio...",3,2023-07-06 16:57:43


In [10]:
sent = SentimentIntensityAnalyzer()
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in df['review_description']]
df['sentiment_score'] = polarity
df.head()

,source,review_description,rating,review_date,sentiment_score
0,Google Play,Meh. Not the greatest experience on a Chromebo...,2,2023-07-08 14:18:24,-0.71
1,Google Play,Pretty good for a first launch!! Its easy to u...,3,2023-07-19 20:52:48,0.99
2,Google Play,"For a brand new app, it's very well optimized....",3,2023-07-06 23:03:11,0.91
3,Google Play,"Great app with a lot of potential! However, th...",3,2023-07-10 00:53:25,0.51
4,Google Play,"The app is good, but it needs a lot of functio...",3,2023-07-06 16:57:43,0.89


In [11]:
lst=[]
for i in range(len(df)):
    if df['sentiment_score'].iloc[i] >0.4:
        lst.append('positive') ##positive
    elif df['sentiment_score'].iloc[i] <-0.4:
        lst.append('negative') # negative
    else:
        lst.append('neutral') #neutral
        

In [12]:
df['sentiment']=lst

In [13]:
df['sentiment'].value_counts()

positive    15111
neutral     14567
negative     3231
Name: sentiment, dtype: int64

In [14]:
corpus = []
for i in tqdm(range(0, len(df))):
    rev = re.sub('[^a-zA-Z0-9]', ' ',str(df['review_description'].iloc[i]))
    rev = rev.lower()
    rev = rev.split()
    
    rev = [lm.lemmatize(word) for word in rev if not word in stopwords.words('english')]
    rev = ' '.join(rev)
    corpus.append(rev)
    
df['cln'] = corpus

100%|███████████████████████████████████████████████████████████████████████████| 32909/32909 [01:09<00:00, 476.90it/s]


In [26]:
df.head()

,source,review_description,rating,review_date,sentiment_score,sentiment,cln,txt_len
0,Google Play,Meh. Not the greatest experience on a Chromebo...,2,2023-07-08 14:18:24,-0.71,negative,meh greatest experience chromebook seems custo...,291
1,Google Play,Pretty good for a first launch!! Its easy to u...,3,2023-07-19 20:52:48,0.99,positive,pretty good first launch easy use self explana...,330
2,Google Play,"For a brand new app, it's very well optimized....",3,2023-07-06 23:03:11,0.91,positive,brand new app well optimized however missing q...,306
3,Google Play,"Great app with a lot of potential! However, th...",3,2023-07-10 00:53:25,0.51,positive,great app lot potential however lot need fixed...,291
4,Google Play,"The app is good, but it needs a lot of functio...",3,2023-07-06 16:57:43,0.89,positive,app good need lot functionality example search...,268


In [16]:
txt_len=[]
for i in df['cln']:
    txt_len.append(len(i))

In [17]:
df['txt_len']=txt_len

In [23]:
df['txt_len']=df['txt_len'].astype(str)
df['rating'] = df['rating'].astype(str)

In [24]:
df['cln']=df['cln'] + df['txt_len'] +df['rating']

In [25]:
cv=CountVectorizer(binary=True)

In [27]:
arr=cv.fit_transform(df['cln']).toarray()

In [28]:
X=arr
y=df['sentiment']

In [29]:
from sklearn.model_selection import train_test_split,cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=0)

In [30]:
model = LogisticRegression().fit(X_train, y_train)
pred=model.predict(X_test)

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.8101894054492049

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test , pred))

              precision    recall  f1-score   support

    negative       0.72      0.47      0.57       958
     neutral       0.76      0.88      0.82      4381
    positive       0.89      0.81      0.85      4534

    accuracy                           0.81      9873
   macro avg       0.79      0.72      0.75      9873
weighted avg       0.81      0.81      0.81      9873



In [33]:
cv=cross_val_score(model , X ,y,cv=3)

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

In [34]:
cv

array([0.75296263, 0.81613491, 0.81238034])

In [35]:
cv.mean()

0.7938259611165179